### READ ME

This file 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
class StudentCourseResults:
    def __init__(self, course_name,data):
        self.course_id = course_name
        self.blok = None
        
        if course_name in data:
            self.blok = data[course_name]

        self.endgrade = None
        self.endgrade_standardized = None        
        self.first_try_grade = None
        self.first_try_standardized = None        
        self.resit_grade = None
        self.resit_standardized = None    

        #dummies
        self.passed_Course = None


        self.firstTry_passed=None
        self.firstTry_present=None
        self.both_notPresent=None
        
        self.absent =0


    def set_endgrade(self, endgrade, grade_standardized):
        if np.isnan(endgrade):
            endgrade = None  
        self.endgrade = endgrade

        if np.isnan(grade_standardized):
            grade_standardized = None 
        self.endgrade_standardized = grade_standardized



    def set_first_try_grade(self, grade,grade_standardized):
        if np.isnan(grade):
            grade = None
        self.first_try_grade = grade

        if np.isnan(grade_standardized):
            grade_standardized = None 

        self.first_try_standardized = grade_standardized


    def set_resit_grade(self, grade,grade_standardized):
        if np.isnan(grade):
            grade = None  
        self.resit_grade = grade

        if np.isnan(grade_standardized):
            grade_standardized = None 
        self.resit_standardized = grade_standardized

    def create_dummies(self):
        self.passed_Course = False


        self.firstTry_passed = False
        self.firstTry_present = False
        self.both_notPresent = False

        if self.endgrade is None:
            self.both_notPresent = True
            return


        if self.endgrade >= 5.5:
            self.passed_Course = True
        
        if self.first_try_grade != None:
            self.firstTry_present = True       





        #Of te wel deze data hebben we niet
        if self.blok == None:
            return
        
        elif self.blok == 1:
            """
            self.firstTry_passed = False
            self.firstTry_present = False
            self.both_notPresent = False
            """
            if self.first_try_grade != None:
                self.firstTry_present = True
            else:
                self.absent = 1

            if self.first_try_grade != None and self.resit_grade == None and self.passed_Course ==True:
                self.firstTry_passed = True
                self.firstTry_present = True                

            if self.first_try_grade == None and self.resit_grade == None:
                self.both_notPresent = True
                self.absent = 2

        elif self.blok == 2:
            """
            self.firstTry_passed = False
            self.firstTry_present = False
            """
            if self.first_try_grade != None:
                self.firstTry_present = True    
                #dit moet dus gefixed worden
                if self.first_try_grade >= 5.5:
                    self.firstTry_passed= True 
            else:
                self.absent = 1
        else:
            """
            self.firstTry_passed = False
            self.firstTry_present = False

            resit_grade

            """
            if self.resit_grade != None:
                self.firstTry_passed= False 
            elif self.resit_grade == None and self.passed_Course == True:
                self.firstTry_passed=True      

            


    def __print__(self):
        string = "mooi man"
        return string
    
    def for_df_column(self):
        
        """
        self.endgrade = None
        self.endgrade_standardized = None        
        self.first_try_grade = None
        self.first_try_standardized = None        
        self.resit_grade = None
        self.resit_standardized = None  
        """
        courses_data = {}
        
        if(self.passed_Course == None):
            return courses_data

        if self.blok == 1:
            courses_data = {
                "1_"+self.course_id + "_Endgrade": self.endgrade,
                "1_"+self.course_id + "_Endgrade_standardized": self.endgrade_standardized,
                "1_"+self.course_id + "_PassedCourse": int(self.passed_Course),
                "1_"+self.course_id + "_Both_notPresent": int(self.both_notPresent),
                "1_"+self.course_id + "_Absent": self.absent,


                #Firsttry
                "1_"+self.course_id + "_FirstTryPassed": int(self.firstTry_passed),
                "1_"+self.course_id + "_FirstTryPresent": int(self.firstTry_present),
                "1_"+self.course_id + "_FirstTryGrade": self.first_try_grade,    
                "1_"+self.course_id + "_FirstTryGrade_standardized": self.first_try_standardized,    









            }         
        elif self.blok == 2:
            courses_data = {


                "2_"+self.course_id + "_Endgrade": self.endgrade,
                "2_"+self.course_id + "_Endgrade_standardized": self.endgrade_standardized,            
                "2_"+self.course_id + "_FirstTryPassed": int(self.firstTry_passed),
                "2_"+self.course_id + "_FirstTryPresent": int(self.firstTry_present),
                "2_"+self.course_id + "_FirstTryGrade": self.first_try_grade,    
                "2_"+self.course_id + "_FirstTryGrade_standardized": self.first_try_standardized,    
                "2_"+self.course_id + "_Absent": self.absent,

                "2_"+self.course_id + "_Both_notPresent": int(self.both_notPresent),                     
                "2_"+self.course_id + "_PassedCourse": int(self.passed_Course),
            }   
        else:
            courses_data = {
                str(self.blok)+"_"+self.course_id + "_Endgrade": self.endgrade,
                str(self.blok)+"_"+self.course_id + "_Endgrade_standardized": self.endgrade_standardized,             
                str(self.blok)+"_"+self.course_id + "_PassedCourse": int(self.passed_Course), 
                str(self.blok)+"_"+self.course_id + "_Both_notPresent": int(self.both_notPresent),
     
            }



        return courses_data


def CreateCourseData(df,data):
  # Create an empty list to store DataFrames for each row
    new_dfs = []

    # Iterate over each row in the original DataFrame
    for i in range(len(df)):
        courses = {}
        # Iterate over each column in the DataFrame
        for column in df.columns:
            # Check if the column represents a course
            if "-standardized" in column:
                continue


            if column.startswith('Course'):
                # Extract the course name
                course_name = column.split('-')[0]
                # Check if the course object exists in the dictionary
                if course_name not in courses:
                    # If not, create a new Course object
                    course = StudentCourseResults(course_name,data)
                    courses[course_name] = course
                else:
                    # If it exists, retrieve the existing Course object
                    course = courses[course_name]

                # Get the endgrade for the current course from the DataFrame
                grade = df.loc[i, column]
                grade_standardized = df.loc[i, column+"-standardized"]

                # Set the endgrade, first try grade, or resit grade based on the column name
                if '-1' in column:
                    course.set_first_try_grade(grade,grade_standardized)

                elif '-R' in column:
                    course.set_resit_grade(grade,grade_standardized)
                else:
                    course.set_endgrade(grade,grade_standardized)

        # Create an empty dictionary to store the data for the current row
        row_data = {}
        
        # Populate the dictionary with data for the current row
        for course_name, course in courses.items():
            course.create_dummies()
            row_data.update(course.for_df_column())
        # Create a DataFrame from the row data
        new_df = pd.DataFrame([row_data])

        # Append the new DataFrame to the list
        new_dfs.append(new_df)




    # Concatenate all DataFrames in the list to create the final DataFrame
    final_df = pd.concat(new_dfs, ignore_index=True)


    endgrade_columns = sorted([col for col in final_df.columns if "_Endgrade" in col])

    other_columns = [col for col in final_df.columns if col not in endgrade_columns]
    print(endgrade_columns)
    new_column_order = endgrade_columns + other_columns

    final_df = final_df[new_column_order]


    # Filter columns in df that do not contain the substring 'Course'
    non_course_columns_df = df.loc[:, ~df.columns.str.contains('Course')]

    # Concatenate the non-course columns from the original DataFrame with the final DataFrame
    merged_df = pd.concat([non_course_columns_df, final_df], axis=1)

    return merged_df



In [ ]:
def AddStandardized(df):
    course_columns = [col for col in df.columns if col.startswith("Course")]
    new_columns = {}
    for col in course_columns:
        mean = df[col].mean(skipna=True)
        std = df[col].std(skipna=True)
        name = f"{col}-standardized"
        new_columns[name]= (df[col]-mean) /std
    
    df = df.assign(**new_columns)

    return df

In [ ]:
def MakeStory(df):
    standardized_columns = [col for col in df.columns if col.endswith("_standardized")]
    standardized_columnsB1B2 = [col for col in df.columns if col.endswith("_standardized") and (col.startswith("1") or (col.startswith("2") and not col.endswith("Endgrade_standardized")))]
    standardized_columnsAfter = [item for item in standardized_columns if item not in standardized_columnsB1B2]

    df["OverAll"] = df[standardized_columns].mean(axis=1)
    df["B1B2"] = df[standardized_columnsB1B2].mean(axis=1)
    df["TheRest"] = df[standardized_columnsAfter].mean(axis=1)
    return df

In [ ]:
def MakeStory2(df):
    columns1 = [col for col in df.columns if (col.endswith("_Absent"))]
    df['Total_absent'] = df[columns1].sum(axis=1)

    columns2 = [col for col in df.columns if (col.startswith(("1","2")) and col.endswith("_FirstTryPassed"))]
    df['Roel_Resit_Factor'] = df[columns2].sum(axis=1)

    df = df.drop(columns=columns1)
    return df

In [ ]:
def run(name,df,data):
    df_1 = AddStandardized(df)
    merged_df = CreateCourseData(df_1,data)

    final_df = MakeStory(merged_df)
    final_df = MakeStory2(final_df)

    # Save the merged DataFrame to an Excel file
    final_df.to_excel(f'{name}.xlsx', index=False)

    # Print a message to confirm the file has been saved
    print("Merged data has been saved to 'merged_data.xlsx")





In [ ]:
CourseBlockData = [
    #Program1
    {
    "Course3" : 1,
    "Course9" : 1, 
    "Course7" : 2, 
    "Course8" : 2,
    "Course16" : 3,

    "Course22" : 4, 
    "Course23" : 4, 
    "Course24" : 5, 
    "Course25" : 5,     
    "Course26" : 6,
    },
    #Program2
    {
    "Course1" : 1,
    "Course11" : 1, 
    "Course2" : 2, 
    "Course10" : 2,
    "Course28" : 3,

    "Course13" : 4, 
    "Course27" : 4, 
    "Course15" : 5, 
    "Course14" : 5,     
    "Course29" : 6,
    },
    #Program3
    {
    "Course1" : 1,
    "Course12" : 1, 
    "Course2" : 2, 
    "Course10" : 2,
    "Course30" : 3,

    "Course13" : 4, 
    "Course27" : 4, 
    "Course31" : 5, 
    "Course14" : 5,     
    "Course29" : 6,
    },
    #Program4
    {
    "Course3" : 1,
    "Course4" : 1, 
    "Course6" : 2, 
    "Course5" : 2,

    "Course16" : 3,

    "Course17" : 4, 
    "Course18" : 4, 
    "Course20" : 5,     
    "Course19" : 5,
    "Course21" : 6, 
    },
]


In [ ]:
for i in range(4):
    #Train data
    df = pd.read_excel('BSA-dataset_2324-Predict_stud.xlsx', sheet_name=i)
    run(f"Pred_Program{i+1}_23_24",df,CourseBlockData[i])